In [0]:
%sql
USE CATALOG conecta;
USE SCHEMA alarmes_aju;

## 1. Número de alarmes por altura de poste.

Os pontos de serviço (postes) com altura superior ou igual a 13 metros, são considerados "altos", pois os caminhões que a empresa tem disponíveis alcançam uma altura máxima inferior a 13 metros.
Essa pergunta serve para que o gerente possa saber a diferença entre alarmes que são em postes baixos e altos. Além disso, é necessário filtrar apenas os alarmes Sem Comunicação e Falha no Driver, pois esses são os que impactam necessariamente essa análise.

Na consulta é utilizado SELECT DISTINCT com IDPontoServico, que é a chave natural de PontoServico. Isso significa que só estamos pegando um registro por poste. Caso um poste possua mais que uma LCU com os mesmos tipos de alarme, será retornado apenas um registro. Isso é interessante, pois postes alto tendem a possuir 4 LCUs, o que iria inflar muito o resultado a favor dos postes altos.

In [0]:
%sql
CREATE OR REPLACE VIEW viewAlarmePontoServico AS (
  SELECT NomeAlarme, TipoPoste, IDPontoServico, Data, TrimestreContrato, Bairro
  FROM fatoalarme
  LEFT JOIN dimtipoalarme ON fatoalarme.AlarmeKey = dimtipoalarme.AlarmeKey
  LEFT JOIN dimpontoservico ON fatoalarme.PontoServicoKey = dimpontoservico.PontoServicoKey
  LEFT JOIN dimdata ON fatoalarme.DataKey = dimdata.DataKey
  WHERE NomeAlarme IN ('Sem Comunicação', 'Falha no Driver')
);

In [0]:
%sql
SELECT NomeAlarme, TipoPoste, count(DISTINCT IDPontoServico) NumeroPostes
FROM viewAlarmePontoServico
WHERE data = (SELECT MAX(Data) FROM dimdata)
GROUP BY NomeAlarme, TipoPoste
ORDER BY TipoPoste;

NomeAlarme,TipoPoste,NumeroPostes
Sem Comunicação,Poste Alto,20
Falha no Driver,Poste Alto,22
Sem Comunicação,Poste Baixo,29
Falha no Driver,Poste Baixo,7


### Discussão
O número de alarmes em pontos de serviço baixo é semelhante ao de poste alto. Isso indica que a situação atual é de que temos aproximadamente o mesmo número de ocorrências de poste baixo e poste alto, evidenciando a necessidade de alugar o caminhão de alcance maior para dar manutenção nos pontos de poste alto.

Sabemos que provavelmente o número de LCUs em poste alto será maior, pois como informado no enunciado, postes altos tendem a possuir 4 LCUs, existindo a possibilidade de que mais do que uma LCU possua alarmes.

## 2. Número de Alarmes no trimestre do contrato atual por altura do poste.

Da mesma forma que na pergunta anterior, estamos interessados no número de alarmes em IDs Ponto Servico distintos, então será utilizado a visão viewAlarmePontoServico como base da consulta.

In [0]:
%sql
SELECT NomeAlarme, TipoPoste, count(DISTINCT IDPontoServico) NumeroPostes
FROM viewAlarmePontoServico
WHERE TrimestreContrato IN (
  SELECT TrimestreContrato
  FROM viewAlarmePontoServico
  WHERE Data = (SELECT MAX(Data) FROM dimdata)
)
GROUP BY NomeAlarme, TipoPoste
ORDER BY TipoPoste;

NomeAlarme,TipoPoste,NumeroPostes
Sem Comunicação,Poste Alto,57
Falha no Driver,Poste Alto,33
Sem Comunicação,Poste Baixo,194
Falha no Driver,Poste Baixo,29


### Discussão
Claramente a maior proporção de alarmes no trimestre atual é de postes baixos. Isso revela uma tendência de que a maior parte dos serviços são para postes baixos.

A proporção de postes altos em relação ao total de postes com sistema de telegestão é de 17 %. O resultado dessa consulta mostra que 29 % dos alarmes no trimestre foram em postes altos, revelando um aumento proporcional de ocorrências em postes altos.

## 3. Número de alarmes no último ETL

É interessante saber o número de alarmes do fechamento do dia anterior - o último ETL.

In [0]:
%sql
CREATE OR REPLACE VIEW viewAlarmeLCU AS (
  SELECT NomeLCU, NomeAlarme, Data, TrimestreContrato, IDLCU, Latitude, Longitude
  FROM fatoalarme a
  LEFT JOIN dimlcu ON dimlcu.LCUKey = a.LCUKey
  LEFT JOIN dimdata ON dimdata.DataKey = a.DataKey
  LEFT JOIN dimtipoalarme ON dimtipoalarme.AlarmeKey = a.AlarmeKey
)

In [0]:
%sql
SELECT NomeAlarme, count(NomeAlarme) NumeroAlarmes
FROM viewalarmelcu
WHERE Data = (SELECT MAX(Data) FROM dimdata)
GROUP BY NomeAlarme
ORDER BY NumeroAlarmes DESC;

NomeAlarme,NumeroAlarmes
Sem Comunicação,53
Acesa Durante o Dia,49
Falha no Driver,31
Apagada Durante a Noite,29
Tensão Alta,10
Tensão Baixa,5


### Discussão
A consulta serve apenas para evidenciar o número de alarmes por Nome de Alarme na data de última carga de dados.

## 4. Número de alarmes resolvidos no trimestre do contrato.

Foi utilizada a heurística de que se um alarme não surge novamente em dada LCU, em um período arbitrário, significa que o defeito foi resolvido.
Exemplo:
LCU 1000 no dia 10/10/2025 apresenta alarme Falha no Driver. Se no dia 11/10/2025 e 12/10/2025 não apareceu mais alarmes de Falha no Driver na LCU 1000, será considerado que o problema foi resolvido.
Essa margem é utilizada, pois existem alarmes que surgem em intervalos curtos de tempo por instabilidade na rede LoRaWan ou rede de energia. Além disso, alarmes realmente resolvidos tendem a não reaparecer por bastante tempo.

O seguinte código originalmente respondia a pergunta 5. Minha resposta da pergunta 5 era muito mal otimizada (código disponível na pergunta 5) e utilizei o Gemini para me ajudar a criar uma consulta baseada somente em SQL. Percebi que com algumas pequenas modificações eu poderia criar uma View para responder a pergunta 4 e 5. Também aprendi a lógica aplicada por esse SQL do Gemini e consegui criar consultas até mais complexas nas próximas perguntas, sem ajuda.

In [0]:
%sql
CREATE OR REPLACE VIEW viewAlarmesResolvidos AS (
  SELECT v1.*
  FROM viewalarmelcu v1
  WHERE v1.Data <= date_sub((SELECT MAX(Data) FROM dimdata), 2)
  -- Só consigo ter certeza que um alarme está resolvido tendo dados de dois dias posteriores à análise.
  AND TrimestreContrato IN (
    SELECT TrimestreContrato
    FROM viewalarmelcu
    WHERE Data = (SELECT MAX(Data) FROM dimdata)
  )
  AND NOT EXISTS (
    SELECT 1
    FROM viewalarmelcu v2
    WHERE v1.NomeAlarme = v2.NomeAlarme
    AND v1.IDLCU = V2.IDLCU
    AND v2.Data > v1.Data
    AND v2.Data <= date_add(v1.Data, 2)
  )
)

In [0]:
%sql
SELECT NomeAlarme, count(NomeAlarme) NumeroAlarmesResolvidos
FROM viewAlarmesResolvidos
GROUP BY NomeAlarme
ORDER BY NumeroAlarmesResolvidos DESC;

NomeAlarme,NumeroAlarmesResolvidos
Sem Comunicação,258
Acesa Durante o Dia,182
Apagada Durante a Noite,73
Falha no Driver,45
Tensão Baixa,3


### 4.2 LCUs e Alarmes distintos resolvidos no trimestre do contrato.

In [0]:
%sql
SELECT NomeAlarme, count(DISTINCT NomeAlarme, NomeLCU) NumeroAlarmesResolvidos
FROM viewAlarmesResolvidos
GROUP BY NomeAlarme
ORDER BY NumeroAlarmesResolvidos DESC;

NomeAlarme,NumeroAlarmesResolvidos
Sem Comunicação,247
Acesa Durante o Dia,146
Apagada Durante a Noite,54
Falha no Driver,37
Tensão Baixa,3


### Discussão
O número elevado de alarmes de Sem Comunicação solucionados é uma consequência direta da atmosfera da cidade de Aracaju que é uma das cidades do Brasil com maior maresia. Esses alarmes ocorrem por conta que o cabo de 1,5 mm² que alimenta as luminária LEDs é partido por causa do salitre, deixando a luminária sem energia e, consequentemente, a LCU fica sem comunicação.

Em segundo lugar aparecem os alarmes de Acesa Durante o Dia. Esses alarmes, na verdade, não foram resolvidos por equipes de manutenção. Ocorre um problema no sistema de telegestão em que as LCUs comandam as luminárias para ficarem acesas durante o dia. Essa noção que o problema é da LCU foi descoberta após a solução da pergunta 8 desse notebook. Até o momento não sabíamos que acontecia esse tipo de situação com as LCUs.

O número de soluções de alarmes Falha no Driver é bem próximo do número de luminárias LED que removemos do campo por estarem queimadas. Isso indica forte relação entre o alarme com um defeito físico da luminária em que a LCU está instalada.

## 5. Número de alarmes resolvidos por dia do Trimestre do Contrato.

Essa pergunta é bem semelhante a anterior e pode ser resolvida pela mesma View, inclusive. Porém, quando a pergunta foi criada eu não tinha noção alguma de como poderia responder a pergunta.

O código a seguir foi minha solução para o problema. Coloquei no Gemini para me trazer uma versão apenas em SQL e ele me trouxe um código SQL como resultado. Aprendi o método do Gemini e vou utilizar no meu trabalho.

A ideia original era utilizar um for loop do python para iterar em todas as datas de DimData. Com a ajuda do Gemini, tive a noção que poderia utilizar tabela virtual para fazer justamente um processo que tem resultado equivalente ao for loop do python, porém muito mais eficiente.

In [0]:
%skip
import pyspark.pandas as ps
dias_trimestre = spark.sql(
    '''
    SELECT DISTINCT Data FROM viewalarmelcu
    WHERE Data < last_dt
    AND TrimestreContrato IN (
        SELECT TrimestreContrato
        FROM viewalarmelcu
        WHERE Data = last_dt
    )
    ''')
dias_trimestre = ps.DataFrame(dias_trimestre)['Data'].sort_values()
result_sql = None
for dia_ in dias_trimestre.to_numpy():
    spk_result = spark.sql(
        f'''
        SELECT Data, Alarme, count(DISTINCT IDLCU, Alarme) as NumeroAlarmesResolvidos
        FROM viewalarmelcu
        WHERE
        (IDLCU, Alarme) NOT IN (
        SELECT IDLCU, Alarme
        FROM viewalarmelcu
        WHERE Data > "{dia_}"
        AND Data <= date_add("{dia_}", 5)
        )
        AND Data = "{dia_}"
        AND Alarme IN ('Sem Comunicação', 'Falha no Driver')
        GROUP BY Data, Alarme
        ORDER BY Data ASC, numeroalarmesresolvidos DESC;
        '''
    )
    if result_sql is None:
        result_sql = spk_result
    else:
        result_sql = result_sql.union(spk_result)
display(result_sql)

In [0]:
%sql
SELECT Data, NomeAlarme, count(IDLCU) AlarmesResolvidos
FROM viewalarmesresolvidos
GROUP BY Data, NomeAlarme
ORDER BY Data, alarmesresolvidos DESC;

Data,NomeAlarme,AlarmesResolvidos
2025-12-01,Acesa Durante o Dia,16
2025-12-01,Sem Comunicação,11
2025-12-01,Falha no Driver,2
2025-12-01,Apagada Durante a Noite,1
2025-12-02,Sem Comunicação,17
2025-12-02,Acesa Durante o Dia,15
2025-12-02,Apagada Durante a Noite,4
2025-12-02,Falha no Driver,4
2025-12-03,Sem Comunicação,29
2025-12-03,Acesa Durante o Dia,12


### Discussão
Além do benefício de sabermos quantos alarmes por dia resolvemos, podemos visualizar uma situação de solução de problema em comandos elétricos, indicado pelo alto número de soluções de alarmes Sem Comunicação.
Por exemplo, no dia 12/12, o número de 53 LCUs sem comunicação é explicado, em grande parte, por conta de um comando elétrico com 48 LCUs que ficou fora de serviço. Essas 48 LCUs ficaram sem energia - sem comunicação - e no dia 12/12 resolvemos o problema.

## 6. LCUs com maior frequência de alarmes nos últimos 30 dias.

Será realizada essa consulta considerando apenas os alarmes Sem Comunicação e Falha no Driver que são os de maior interesse para a gerência.

In [0]:
%sql
SELECT NomeLCU, NomeAlarme, count(DISTINCT NomeLCU, Data) NumeroAlarmes, numeroalarmes / 30 * 100 Frequencia
FROM viewalarmelcu
WHERE NomeAlarme IN ('Sem Comunicação', 'Falha no Driver')
AND Data > date_sub((SELECT MAX(Data) FROM dimdata), 30)
GROUP BY NomeLCU, NomeAlarme
ORDER BY NumeroAlarmes DESC
LIMIT 50;

NomeLCU,NomeAlarme,NumeroAlarmes,Frequencia
LCU 602338,Sem Comunicação,30,100.0
LCU 6002C7,Sem Comunicação,30,100.0
LCU 1111,Falha no Driver,30,100.0
LCU 1545,Sem Comunicação,30,100.0
LCU 503212,Sem Comunicação,30,100.0
LCU 603325,Sem Comunicação,30,100.0
LCU 604CB6,Falha no Driver,30,100.0
LCU 26EC,Sem Comunicação,30,100.0
LCU 602494,Sem Comunicação,30,100.0
LCU 33FF,Sem Comunicação,30,100.0


### Discussão
A consulta mostra quais LCUs estão com maior frequência de alarmes, considerando dados dos últimos 30 dias. As LCUs no topo do resultado devem possuir prioridade para resoluções de problemas por parte da operação.
Esses dados servem para que a gerência cobre uma atitude da operação em relação a esses alarmes.

A maior parte desses alarmes, entretanto, são de situações com impossibilidade. Ou seja, dependemos de atuação de terceiros para que uma providência possa ser tomada.

## 7. LCUs com maior frequência de alarme Aceso Durante o Dia nos últimos 30 dias.

Temos um problema com o alarme de LED Aceso Durante o Dia, pois aparentemente o próprio sistema de telegestão está gerando esse defeito, deixando algumas LEDs da cidade acesas durante o dia. Quando uma equipe faz a intervenção e troca o dispositivo IoT, muitas vezes o dispositivo está apagando normalmente ao excitar o relé, em testes de bancada.
Percebemos que após alguns dias com o defeito a LED em campo apaga e o dispositivo IoT LCU para de gerar alarmes. Talvez, então, vários dias seguidos com esse alarme indique que realmente existe um defeito e não é algo transitório.

In [0]:
%sql
SELECT NomeLCU, count(NomeLCU) NumeroAlarmes, numeroalarmes / 30 * 100 Frequencia
FROM viewalarmelcu
WHERE NomeAlarme = 'Acesa Durante o Dia'
AND Data > date_sub((SELECT MAX(Data) FROM dimdata), 30)
GROUP BY NomeLCU
ORDER BY NumeroAlarmes DESC
LIMIT 50;

NomeLCU,NumeroAlarmes,Frequencia
LCU 2C38,30,100.0
LCU 6008AA,30,100.0
LCU 195D,30,100.0
LCU 13FE,29,96.66666666666667
LCU 1CE5,29,96.66666666666667
LCU 1375,28,93.33333333333333
LCU 2DC8,28,93.33333333333333
LCU 1405,28,93.33333333333333
LCU 1849,28,93.33333333333333
LCU 108A,28,93.33333333333333


### Discussão
Essa pergunta é uma consequência do elevado número de alarmes detectados na pergunta 4. Nessa consulta a tentativa era de encontrar casos em que o alarme de Acesa Durante o Dia era permanente e não transitório, gerado por algum mecanismo na LCU. A ideia é que alarmes com alta frequência talvez indiquem que o alarme é permanente.

Entretanto, avaliando essas amostras, percebemos casos de LCUs que passaram vários dias emitindo alarme e simplesmente voltaram a funcionar normalmente depois.
A pergunta a seguir foi criada para detectar esses casos.

## 8. LCUs com dias consecutivos com alarme Acesa Durante o Dia.

Apesar dessa pergunta ser parecida com a anterior essa é muito mais importante para a gerência e o propósito é diferente em duas situações:
1. Os dias com alarme precisam ser consecutivos para a contagem - vou estabelecer mínimo de 3 dias.
2. O alarme deve ter parado em um momento posterior. Vou novamente considerar um período de 2 dias sem alarmes do tipo analisado.

Exemplo: LCU 1000 gera alarme de Aceso Durante o Dia na Data 10/11/2025. Ela deve emitir o alarme também nas datas 11/11/2025 e 12/11/2025. Além disso, o alarme não pode ser emitido nas datas 13/11/2025 e 14/11/2025. O período de dois dias em que o alarme não é emitido é para termos certeza que realmente o alarme parou e o problema não persiste.

A LCU pode ficar mais do que três dias consecutivos com alarme para um resultado ser retornado nessa consulta. No exemplo acima, a LCU 1000 poderia ter iniciado a emitir alarme no dia 01/11/2025, mas deve seguir as regras estabelecidas acima.

Essa pergunta serve para evidenciar à empresa responsável pelo sistema de Telegestão a instabilidade do alarme de Acesa Durante o Dia. As LCUs que apresentam esse alarme ligam a luminária LED durante o dia de forma errada, por vários dias consecutivos e, sem intervenção de equipe de manutenção, a LCU volta a funcionar normal, desligando a lâmpada.
Essa pergunta servirá como evidência em relatório.

*Update. No dia 04/12/2025 foi enviado relatório com as LCUs detectadas por esse trabalho à empresa responsável pela Telegestão. A empresa reconheceu que realmente existe um problema e atualmente estão entrando em contato com o fabricante do equipamento para esclarecimentos.

In [0]:
%sql
CREATE OR REPLACE VIEW viewAcesaDiaConsecutivas AS (
  SELECT v1.*
  FROM viewalarmelcu v1
  WHERE v1.Data <= date_add((SELECT MAX(Data) FROM dimdata), - 4)
  -- v1.Data deve ser tal que garanta existir pelo menos duas datas após os três dias consecutivos, para a clásula AND NOT EXISTS funcionar.
  -- Caso não exista pelo menos duas datas após os três dias avaliados no banco de dados, NOT EXISTS poderá ser avaliado como True indevidamente,
  -- fazendo com que traga como resposta LCU que não deixaram de emitir alarme Acesa Durante o Dia.
  AND v1.NomeAlarme = 'Acesa Durante o Dia'
  AND v1.NomeLCU IN (
    SELECT v2.NomeLCU
    FROM viewalarmelcu v2
    WHERE v1.IDLCU = v2.IDLCU
    AND v2.NomeAlarme = 'Acesa Durante o Dia'
    AND v2.Data < date_add(v1.Data, 3)
    AND v2.Data >= v1.Data
    GROUP BY v2.NomeLCU
    HAVING count(v2.NomeLCU) = 3
  )
  AND NOT EXISTS (
    SELECT 1
    FROM viewalarmelcu v2
    WHERE v1.IDLCU = v2.IDLCU -- Mesma LCU
    AND v1.NomeAlarme = v2.NomeAlarme -- Mesmo Alarme
    AND v2.Data IN (date_add(v1.Data, 3), date_add(v1.Data, 4))
    -- As vezes o sistema de telegestão não comunica um alarme em um dia, por isso o check em dois dias.
  )
)

In [0]:
%sql
SELECT *
FROM viewAcesaDiaConsecutivas
LIMIT 50;

NomeLCU,NomeAlarme,Data,TrimestreContrato,IDLCU,Latitude,Longitude
LCU 102C,Acesa Durante o Dia,2025-11-24,T16,2543196311596,-11.007403,-37.07467
LCU 10FD,Acesa Durante o Dia,2025-11-07,T16,2543196311805,-10.926409,-37.052948
LCU 11F0,Acesa Durante o Dia,2025-11-21,T16,2543196312048,-11.005013,-37.10332
LCU 12B0,Acesa Durante o Dia,2025-11-23,T16,2543196312240,-11.007684,-37.074886
LCU 12BD,Acesa Durante o Dia,2025-11-12,T16,2543196312253,-11.004372,-37.072433
LCU 12C3,Acesa Durante o Dia,2025-11-28,T16,2543196312259,-10.927903,-37.04331
LCU 1313,Acesa Durante o Dia,2025-11-27,T16,2543196312339,-11.00379,-37.071953
LCU 151A,Acesa Durante o Dia,2025-11-21,T16,2543196312858,-10.993076,-37.062847
LCU 1581,Acesa Durante o Dia,2025-11-27,T16,2543196312961,-10.974787,-37.038902
LCU 1624,Acesa Durante o Dia,2025-11-05,T16,2543196313124,-10.991919,-37.050865


### Discussão
Foi possível detectar mais de 100 casos em que a LCU ficou ao menos três dias consecutivos com alarme de acesa durante o dia. Segue exemplo do gráfico de potência medida ao longo do tempo da LCU 10FD:

![LCU 10FD Day Burner.png](./LCU 10FD Day Burner.png)

Onde a seta está apontando é a data 07/11/2025, que é a data retornada na consulta. Essa data representa o primeiro dia de alarme Aceso Durante o Dia considerado na regra criada de três dias consecutivos, conforme explicado no enunciado. Nas datas 08/11 e 09/11 é possível notar que as potências medidas estão acima de 0 e, a partir do dia 10/11, as potência medidas estão em 0.

Gostaria de deixar claro que não existe maneira de consultar isso no sistema de telegestão, a não ser ir entrando em cada LCU individualmente no sistema e checando o gráfico como a figura acima. Esse trabalho foi graças a esse MVP de DW.

### 8.2 LCUs com alarme Acesa Durante durante apenas um dia

A empresa de telegestão sugeriu que não existiam situações em que a LCU emitia alarme de acesa durante o dia por apenas um único dia e depois voltava ao normal. Fiz essa nova consulta para mostrar que existem, sim, essas situações. Inclusive é muito mais comum do que a situação anterior, de dias consecutivos.

In [0]:
%sql
SELECT v1.*
FROM viewalarmelcu v1
WHERE v1.Data < (SELECT MAX(Data) FROM dimdata)
AND v1.NomeAlarme = 'Acesa Durante o Dia'
AND v1.NomeLCU NOT IN (
  SELECT v2.NomeLCU
  FROM viewalarmelcu v2
  WHERE v2.Data IN (date_add(v1.Data, 1), date_add(v1.Data, - 1))
  AND v1.IDLCU = v2.IDLCU
  AND v2.NomeAlarme = V1.NomeAlarme
)
LIMIT 50;

NomeLCU,NomeAlarme,Data,TrimestreContrato,IDLCU,Latitude,Longitude
LCU 102C,Acesa Durante o Dia,2025-11-15,T16,2543196311596,-11.007403,-37.07467
LCU 1075,Acesa Durante o Dia,2025-11-12,T16,2543196311669,-10.930195,-37.057327
LCU 1181,Acesa Durante o Dia,2025-11-17,T16,2543196311937,-10.935231,-37.08304
LCU 11F4,Acesa Durante o Dia,2025-11-20,T16,2543196312052,-10.976127,-37.066467
LCU 1251,Acesa Durante o Dia,2025-11-12,T16,2543196312145,-10.927792,-37.05294
LCU 12BA,Acesa Durante o Dia,2025-11-12,T16,2543196312250,-10.929392,-37.056816
LCU 12DE,Acesa Durante o Dia,2025-11-12,T16,2543196312286,-10.929697,-37.05679
LCU 12EE,Acesa Durante o Dia,2025-11-17,T16,2543196312302,-10.918912,-37.05501
LCU 1314,Acesa Durante o Dia,2025-11-20,T16,2543196312340,-11.003507,-37.07176
LCU 1337,Acesa Durante o Dia,2025-11-26,T16,2543196312375,-10.93952,-37.080734


### Discussão
Quase 400 LCUs ficaram acesas ao menos um dia, mostrando ao time responsável do sistema de telegestão que esse caso existe e é até mais comum do que o anterior.

## 9. Número de LCUs com Alarmes recorrentes.

Alarmes em dias não consecutivos podem indicar que o alarme foi corrigido/supremido em determinada data e o problema retornou em uma data posterior. Importante para determinar se estamos tendo recorrência de defeitos ou se a LCU em questão está sofrendo instabilidade.

A ideia será retornar registros de LCU que possuem determinado alarme na data D e que não possuem esse mesmo alarme na data D + 1 dia. A contagem dessas LCUs retornará quantos defeitos recorrentes ocorrem por LCU, por alarme.
Nessa pergunta, também queremos detectar alarmes originados por instabilidade. Então o critério será D + 1 dia e não será utilizada a heurística de verificar D + 2 dias como em perguntas anteriores.

In [0]:
%sql
CREATE OR REPLACE VIEW viewAlarmesRecorrentes AS (
  SELECT v1.*
  FROM viewalarmelcu v1
  WHERE v1.Data < (SELECT MAX(Data) FROM dimdata)
  AND NOT EXISTS (
    SELECT 1
    FROM viewalarmelcu v2
    WHERE v1.NomeAlarme = v2.NomeAlarme
    AND v1.IDLCU = v2.IDLCU
    AND v2.Data = date_add(v1.Data, 1)
  )
)

In [0]:
%sql
SELECT NomeLCU, NomeAlarme, count(NomeLCU) NumeroOrrencias
FROM viewalarmesrecorrentes
GROUP BY NomeLCU, NomeAlarme
HAVING NumeroOrrencias > 1 -- Maior que 1 para mostrar apenas recorrentes.
ORDER BY NomeLCU, NumeroOrrencias DESC
LIMIT 50;

NomeLCU,NomeAlarme,NumeroOrrencias
LCU 101F,Apagada Durante a Noite,4
LCU 102C,Acesa Durante o Dia,3
LCU 102C,Sem Comunicação,2
LCU 108A,Acesa Durante o Dia,2
LCU 1265,Sem Comunicação,2
LCU 1294,Acesa Durante o Dia,2
LCU 12B2,Tensão Baixa,3
LCU 12BD,Acesa Durante o Dia,2
LCU 1375,Acesa Durante o Dia,2
LCU 1405,Acesa Durante o Dia,2


In [0]:
%sql
SELECT NomeLCU, count(NomeLCU) NumeroOrrencias
FROM viewalarmesrecorrentes
WHERE NomeAlarme = 'Sem Comunicação'
GROUP BY NomeLCU
HAVING NumeroOrrencias > 1
ORDER BY NumeroOrrencias DESC
LIMIT 50;

NomeLCU,NumeroOrrencias
LCU 604588,5
LCU 60136F,5
LCU 6012C4,4
LCU 6012A1,3
LCU 3768,3
LCU 5039ED,3
LCU 3769,3
LCU 600A14,3
LCU 604578,3
LCU 604B65,3


### Discussão

O caso de alarmes recorrentes é mais útil quando analisamos o número de ocorrências de alarmes Sem Comunicação. Foi mencionado anteriormente que na maioria dos casos esse alarmes é consequência de uma falta de energia que precisa ser reestabelecida pela equipe de manutenção. Pórem, existem casos em que esse alarme surge devido a falhas no sinal da rede LoraWan, que na verdade é o caso em que faz mais sentido com o nome do alarme.

Esses casos de falha no sinal da rede podem ser verificados com essa consulta de recorrências em um intervalo curto de tempo. As LCUs 60136F, 604588 e 6012C4, por exemplo, são praticamente vizinhas umas das outras, indicando que talvez exista uma falha na cobertura nesse local.

## 10. Alarmes por bairro no trimestre do contrato.

In [0]:
%sql
CREATE OR REPLACE VIEW viewConecta AS (
  SELECT IDLCU, NomeLCU, dimlcu.Latitude, dimlcu.Longitude, Data, TrimestreContrato, IDPontoServico, Bairro, MarcoContrato, NomeAlarme
  FROM fatoalarme
  LEFT JOIN dimtipoalarme ON fatoalarme.AlarmeKey = dimtipoalarme.AlarmeKey
  LEFT JOIN dimpontoservico ON fatoalarme.PontoServicoKey = dimpontoservico.PontoServicoKey
  LEFT JOIN dimdata ON fatoalarme.DataKey = dimdata.DataKey
  LEFT JOIN dimlcu ON fatoalarme.LCUKey = dimlcu.LCUKey
);

In [0]:
%sql
SELECT Bairro, NomeAlarme, count(DISTINCT IDLCU) NumeroAlarmes
FROM viewConecta
WHERE TrimestreContrato IN (
  SELECT TrimestreContrato
  FROM viewConecta
  WHERE Data = (SELECT MAX(Data) FROM dimdata)
)
GROUP BY Bairro, NomeAlarme
ORDER BY Bairro, NumeroAlarmes DESC;

Bairro,NomeAlarme,NumeroAlarmes
13 de julho,Sem Comunicação,16
13 de julho,Acesa Durante o Dia,3
13 de julho,Falha no Driver,2
17 de março,Acesa Durante o Dia,3
17 de março,Sem Comunicação,2
17 de março,Falha no Driver,1
18 do forte,Tensão Baixa,1
18 do forte,Falha no Driver,1
Aeroporto,Sem Comunicação,14
Aeroporto,Acesa Durante o Dia,8


### Discussão
Para finalizar, uma consulta para mostrar quantas LCUs distintas por bairro no trimestre do contrato emitiram alarme. Essa consulta é útil para tentar identificar uma região que seja mais afetada por algum tipo de alarme.